In [1]:
%ls

 Volume in drive C has no label.
 Volume Serial Number is 80B8-8562

 Directory of C:\Users\taiseii\OneDrive - TU Eindhoven\Documents\WebA\a3

06-Jan-19  22:18    <DIR>          .
06-Jan-19  22:18    <DIR>          ..
06-Jan-19  18:01    <DIR>          .ipynb_checkpoints
03-Jan-19  14:01           130,291 e3-EMM.pdf
06-Jan-19  18:17         6,145,079 HW3 - Copy.ipynb
06-Jan-19  22:18         6,146,853 HW3.ipynb
03-Jan-19  13:55           638,020 HW3instructions.pdf
04-Jan-19  17:52         2,893,465 J5-EMM_DMKD.pdf
03-Jan-19  15:49       132,546,563 Web_Anlaytics_Nov_2018_final.csv
04-Jan-19  15:15         1,753,030 week3B-SD.pdf
04-Jan-19  14:04         4,516,827 week4A_EMM.pdf
04-Jan-19  17:39        23,047,315 week4B_EMM2.pdf
04-Jan-19  17:39         2,125,837 week4C_SP.pdf
              10 File(s)    179,943,280 bytes
               3 Dir(s)  887,362,396,160 bytes free


In [13]:
import pandas as pd
import numpy as np
import heapq as hp
from collections import deque
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns # also improves the look of plots
import random
import queue
import operator

In [3]:
df = pd.read_csv("Web_Anlaytics_Nov_2018_final.csv",encoding = "ISO-8859-1")

<h1>Q1</h1>

<h2>a)</h2> <h4>targets = Combination id : Converted :</h4>

In [4]:
df.head()
df.tail()
df.shape

(344785, 22)

In [5]:
list(df)

['Screen Resolution',
 'Browser',
 'Browser Version',
 'Device Type',
 'Device',
 'OS',
 'OS Version',
 'User Agent',
 'Traffic Source',
 'Combination Id',
 'Converted',
 'Conversion Time',
 'Returning Visitor',
 'Hit Time',
 'User Language',
 'URL',
 'Referring URL',
 'City',
 'Region',
 'Country',
 'Goal 1 Converted',
 'Goal 1 Converted Time']

In [6]:
for x in list(df):
    header = str(x)
    print(df[header].unique().shape)

(1938,)
(45,)
(834,)
(4,)
(4302,)
(27,)
(173,)
(22336,)
(5,)
(2,)
(2,)
(4456,)
(2,)
(255253,)
(279,)
(42758,)
(20392,)
(15636,)
(2007,)
(228,)
(3,)
(4373,)


In [7]:
df['Converted'].unique()

array(['No', 'Yes'], dtype=object)

In [8]:
df['Conversion Time'].dropna().index

Int64Index([    88,    206,    282,    323,    397,    574,    604,    645,
               690,    810,
            ...
            343074, 343185, 343468, 343500, 343524, 343851, 344052, 344064,
            344485, 344521],
           dtype='int64', length=4470)

<h2>b)</h2><h4>assumption 1 : Data is generated by human</h4><h4>assumption 2 : Human produced this data was in similar state of mind(not under extreme stress/pressure etc)</h4> <h4>assumption 3 : There is no statistical bias(this sample represent the population)</h4> <h4>assumption 4 : website assigned is random</h4> 
<h4>assumption 5 : Hit time = when the page has landed for the first time</h4>
<h4>descriptor 1 : everything excluding targets. Some of the attribute might seem useless at first but might contribute in identifying the relevant subgroup later.</h4> 
<h4>descriptor 2 : "conversion rate for 1 or 2(new variable)"(when later subgrouping people)=(total conversion=true for of 1 or 2)/(total number of 1 or 2 shown). This is a good indicator to see proportion of success</h4>
<h4>descriptor 3 : time to conversion = Hit time - conversion time, if assumption 5 is true then this can be a measurement of how fast the customer has clicked the website which can be a indicator how good the conversion is.</h4>
<h4>preprocessing 0 : df = pd.read_csv("Web_Anlaytics_Nov_2018_final.csv",encoding = "ISO-8859-1") after separating columns using excel. This create one table with both targets and descriptors. We will not separate data at this point since this will be taken care by beam search</h4>
<h4>preprocessing 1 : Goal 1 Converted Time = Conversion Time</h4>
<h4>preprocessing 2 : Create descriptor 3</4>
<h4>preprocessing 3 : Converesion of dataframe to numpy array</h4>

In [9]:
#df['Hit Time']-df['Conversion Time']
type(df['Hit Time'][0])
type(df['Conversion Time'][0])
print(df['Hit Time'][0])
print(df['Conversion Time'][0])
import time
import datetime
import dateutil.parser
# print(dateutil.parser.parser(df['Hit Time'][0]))
datetime.datetime.strptime(df['Hit Time'][0], "%Y-%m-%dT%H:%M:%S")
datetime.datetime.strptime(str(df['Conversion Time'][88]), "%Y-%m-%dT%H:%M:%S")

2018-03-22T08:47:39
nan


datetime.datetime(2018, 3, 22, 13, 47, 24)

In [10]:
df2 = pd.DataFrame()
df2['Browser'] = df['Browser'].astype('category') 
df2['Device Type'] = df['Device Type'].astype('category')
df2['Device'] = df['Device'].astype('category')
df2['OS'] = df['OS'].astype('category')
# df2['User Agent']=df['User Agent'].astype('category')
df2['Returning Visitor']=df['Returning Visitor'].astype('category')
df2['User Language']=df['User Language'].astype('category')
df2['Country']=df['Country'].astype('category')
df2['X']=df['Combination Id'].astype('category')
df2['Y']=df['Converted'].astype('category')

In [16]:
#user agent info = device type .... 
import re
regex = re.compile(r'[^\s]+')
re.search(regex, df['Device'][0]).group(0)

devices = deque()
for x in range(len(df2['Device'])):
#     print(re.search(regex, df2['Device'][x]).group(0))
    devices.append(re.search(regex, df2['Device'][x]).group(0))
    
df2['Device 2'] = devices
# devices

In [ ]:
df2_np = np.asarray(list(zip(df2['Browser'],
                            df2['Device Type'],
                            df2['Device'],
                            df2['OS'],
                            df2['User Agent'],
                            df2['Returning Visitor'],
                            df2['User Language'],
                            df2['Country'],
                            df2['X'],
                            df2['Y'])))

In [ ]:
df2_np_descriptors = np.asarray(list(zip(df2['Browser'],
                            df2['Device Type'],
                            df2['Device'],
                            df2['OS'],
                            df2['User Agent'],
                            df2['Returning Visitor'],
                            df2['User Language'],
                            df2['Country'])))
df2_np_targets = np.asarray(list(zip(df2['X'],
                            df2['Y'])))

In [ ]:
#maximum d
len(df2_np_descriptors[0])

<h1>Q2</h1>

<h2>a)</h2>

In [ ]:
import sys
!{sys.executable} -m pip install pysubgroup

In [ ]:
print(df2_np.shape)
sequence = [[list(),1.0]]
sequence

In [ ]:
# def qualityM(n1,n2,n3,n4):
#     if (n1*n4 + n2*n3) == 0 :
#         yule = 0
#     else :
#         yule = (n1*n4 - n2*n3)/(n1*n4 + n2*n3)
#     return yule

In [ ]:
#df must be a subset(only the solution) of the DataFrame which meatches the descriptor 
def YQ(df):
    if df.shape != (2,2):
        return 0
    else:
        n1 = df[0][0]
        n2 = df[1][0]
        n3 = df[0][1]
        n4 = df[1][1]
        return ((n1*n4) - (n2*n3))/((n1*n4) + (n2*n3))

In [ ]:
def RefinementO(seed, df, possibleValues):
    result = []
    for attribute in df.columns.drop(['X','Y']):
        checked = False
        for condition in seed:
            if attribute == condition[0]:
                checked = True
                break
        if not checked:
            attribute_str = str(attribute)
            if len(df[attribute_str].unique()) == 2 : #binary
                result.append(seed + [(attribute, '=',
                                       possibleValues[attribute][0])])
                result.append(seed + [(attribute, '=',
                                       possibleValues[attribute][1])])
            else: 
                for value in possibleValues[attribute]: #non binary
                    result.append(seed + [(attribute, '=', value)])
                    result.append(seed + [(attribute, '!=', value)])
    return result

In [ ]:
df2.columns.drop(['X','Y'])

In [ ]:
def PossibleValues(dataframe):
     result = {}
     for attribute in dataframe.columns:
         result[attribute] = set()
         for value in dataframe[attribute]:
             result[attribute].add(value)
     return result

In [ ]:
PossibleValues(df2.drop(['X','Y'], axis=1))

In [ ]:
def minsizeOK(descriptor, min_size, dataframe=df):
    descriptor_slice = dataframe
    for condition in descriptor:
        descriptor_slice = descriptor_slice[ops[condition[1]](descriptor_slice[condition[0]], 
                                                              condition[2])]
    if descriptor_slice.shape[0] > min_size:
        return True
    else:
        return False

In [ ]:
def BSearch(df, w, d, qualityM = YQ):
    counter = 0
    candidate = deque()
    candidate.append([])
    resultSet =[]
    possible_descriptors = PossibleValues( df.drop(['X','Y'], axis=1) )
    
    for level in range(d):
        beam = []
        while candidate:
            seeds = candidate.pop()
            
            if len(seeds)>0:
                seed = seeds[2]
            else:
                seed = seeds
            candidateDescriptors = RefinementO(seed, df, possible_descriptors)
            for drs in candidateDescriptors:
                drs_slice = df
                for condition in descriptor:
                    drs_slice = drs_slice[ops[condition[1]]
                                          (descriptor_slice[condition[0]],condition[2])]
                quality = abs((qualityM(drs_slice)))
                if quality == 1.0:
                    continue
                if minsizeOK(descriptor, min_size):
                    if len(result_set) == q:
                        heapq.heappushpop(result_set, (quality, count, descriptor))
                    else:
                         heapq.heappush(result_set, (quality, count,descriptor))
                    if len(beam) == w:
                        heapq.heappushpop(beam, (quality, count, descriptor))
                    else:
                        heapq.heappush(beam, (quality, count, descriptor))
                    counter += 1
        while (beam):
            candidate.append(hp.heappop(beam))
    print(resultSet)
    return 1
                    

In [ ]:
BSearch(df2, 2, 3, qualityM = YQ)

In [ ]:
# def beamSearch(descriptors,targets, w, d):
#     candidate = []
#     resultSet = []
    
#     for i in range(d):
#         beam = []
#         beam = [0]*w
        
#         while len(candidate)>0 :
#             print(i)
#             descriptor_step = []
#             for dr in range(descriptors.shape[0]):
#                 descriptor_step.append(descriptors[dr][i-1])
#             descriptor_step_df = pd.DataFrame(descriptor_step)
#             print(descriptor_step_df[0].unique())
#             targets_df = pd.DataFrame(targets)
            
#             score_d = []
#             for uq in descriptor_step_df[0].unique():
#                 n1 = 0
#                 n2 = 0
#                 n3 = 0
#                 n4 = 0
#                 subgroup = []
#                 subgroup = descriptor_step_df.index[descriptor_step_df[0] == uq].tolist()
#                 for index in range(len(subgroup)):
#                     X = targets_df.loc[subgroup[index],0]
#                     Y = targets_df.loc[subgroup[index],1]
#                     if (X== '1') and (Y=='No'):
#                         n1 += 1
#                     elif (X=='1') and (Y=='Yes'):
#                         n2 += 1
#                     elif (X=='2') and (Y=='No'):
#                         n3 += 1
#                     elif (X=='2') and (Y=='Yes'):
#                         n4 += 1
#                 hp.heappush(score_d, (qualityM(n1,n2,n3,n4), uq))
#             print(hp.nsmallest(5, score_d))
#             print(hp.nlargest(5, score_d))
            
# #             seed = candidate.pop(0)
# #             seets = {seed}
            
#             hp.heappush(resultSet, (hp.nlargest(1, score_d)))
#             hp.heappush(beam, (hp.nlargest(1, score_d)))
#             for element in seets:
                
#                 quality = qualityM(n1,n2,n3,n4)
                
#                 hp.heappush(resultSet, (quality, element))
#                 hp.heappush(beam, (quality, element))
#         while len(beam)>0:
#             candidate.append(beam.pop())
#     return resultSet

In [ ]:
beamSearch(df2_np_descriptors,df2_np_targets,2,2)

In [ ]:
x = deque()
# x.append([])
x
len(x)
if(x):
    print ('yes')

<h5>w is number of descroptions within the d(level), seed is higher level than candidate. It's going to be inside the beam</h5>

In [ ]:
df2_np_targets.shape[0]


In [ ]:
#experimentation zone 2
h = []
hp.heappush(h, (5, 'write code'))
hp.heappush(h, (7, 'write code'))
hp.heappush(h, (6, 'write code'))
hp.heappush(h, (8, 'write code'))
hp.heappush(h, (4, 'write code'))
hp.heappush(h, (7, 'write code'))
hp.heappush(h, (6, 'write code'))
hp.heappush(h, (2, 'write code'))
hp.heappop(h)